In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from datetime import datetime

In [2]:
print("starting batch process......")

print("loading transaction data......")
transaction_file_name = "../ETL/CSV/obd_transaction.csv"
df = pd.read_csv(transaction_file_name, index_col=0)
print("transaction data loaded")
print(df)

starting batch process......
loading transaction data......
transaction data loaded
        InvoiceNumber InvoiceDate CustomerCode  ProductCode  BaseQty  \
0         HSX61C00685  2022-02-01    OWK001831     10000566      6.0   
1         HSA22C00334  2022-02-01    RKP003445     10000512    230.0   
2         HSR25C00344  2022-02-01    PKA008980     10000548      5.0   
3         HSR67C00536  2022-02-01    JTG004975     10000571     24.0   
4         HSM61C00018  2022-02-01    PSM000068     10000604     40.0   
...               ...         ...          ...          ...      ...   
7230273   HSAI1C02522  2022-08-04    SPM001751     10000600     50.0   
7230274   HSA25C03600  2022-08-04    RKP005221     10000600     50.0   
7230275   HSW05C03323  2022-08-04    YSR001143     10000600     50.0   
7230276   HSYY3C02062  2022-08-04    BVD001250     10000600     50.0   
7230277   HSAP5C01879  2022-08-04    NSV008958     10000600     50.0   

         NetAmount  DiscAmount  UnitPrice  \
0     

In [3]:

# transform transaction table
print("transforming transaction data......")
basket_df = df.groupby(['InvoiceNumber', 'ProductCode']).size().unstack(fill_value=0).astype(bool)
print(basket_df)


transforming transaction data......
ProductCode    10000183  10000186  10000187  10000204  10000205  10000213  \
InvoiceNumber                                                               
HS792201004       False     False     False     False     False     False   
HS792201005       False     False     False     False     False     False   
HS792201006       False     False     False     False     False     False   
HS792201007       False     False     False     False     False     False   
HS792201008       False     False     False     False     False     False   
...                 ...       ...       ...       ...       ...       ...   
IVZZ62200297      False     False     False     False     False     False   
IVZZ62200298      False     False     False     False     False     False   
IVZZ62200299      False     False     False     False     False     False   
IVZZ62200300      False     False     False     False     False     False   
IVZZ62200301      False     False     Fa

In [4]:

# create frequent item sets
# rules = minimum support 5%
print("creating frequent itemsets with minimum support of 5%....")
frequent_itemsets  = apriori(basket_df, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
print(frequent_itemsets)

# create association rules
print("creating association rules [support, confidence, lift,  leverage, conviction] with minimum lift of 1 .......")
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values('lift', ascending=False)
print(rules)

# clean rules
print("further cleaning of association rules [confidence >= 0.5, lift >= 1.5]....")
rules = rules[rules['confidence'] >= 0.5]
rules = rules[rules['lift'] >= 1.5]
print(rules)


creating frequent itemsets with minimum support of 5%....
     support                                  itemsets  length
0   0.082677                                (10000302)       1
1   0.075531                                (10000347)       1
2   0.055541                                (10000359)       1
3   0.128298                                (10000378)       1
4   0.157608                                (10000430)       1
..       ...                                       ...     ...
84  0.051753            (10000640, 10000629, 10000600)       3
85  0.083354            (10000602, 10000603, 10000604)       3
86  0.086368            (10000627, 10000628, 10000629)       3
87  0.059957            (10000640, 10000627, 10000628)       3
88  0.065882  (10000600, 10000627, 10000628, 10000629)       4

[89 rows x 3 columns]
creating association rules [support, confidence, lift,  leverage, conviction] with minimum lift of 1 .......
              antecedents           consequents  antec

In [81]:
#  Preparing clean data for Tableau (from frozenset to string) 
print("formatting data....")
rules["antecedents"] = rules["antecedents"].apply(lambda x: ', '.join([str(i) for i in list(x)])).astype("unicode")
rules["consequents"] = rules["consequents"].apply(lambda x: ', '.join([str(i) for i in list(x)])).astype("unicode")

# split antecedents and consequents into separate columns
print("splitting antecedents and consequents into separate columns....")
rules = rules.join(rules['antecedents'].str.split(',', expand=True).add_prefix('antecedent_'))
rules = rules.join(rules['consequents'].str.split(',', expand=True).add_prefix('consequent_'))

# # Explode the consequents column to get the consequents in separate rows
# print("Exploding consequents column....")
# rules = rules.assign(**{'consequents': rules['consequents'].str.split(',')}).explode('consequents')
print(rules)

formatting data....
splitting antecedents and consequents into separate columns....
                      antecedents         consequents  antecedent support  \
179            10000602, 10000604            10000603            0.098043   
178            10000602, 10000603            10000604            0.135761   
183                      10000604  10000602, 10000603            0.152707   
184            10000627, 10000628            10000629            0.142926   
189                      10000629  10000627, 10000628            0.152607   
180            10000603, 10000604            10000602            0.108376   
202            10000600, 10000629  10000627, 10000628            0.117806   
196  10000600, 10000627, 10000628            10000629            0.110560   
185            10000627, 10000629            10000628            0.103019   
197  10000600, 10000627, 10000629            10000628            0.078765   
204            10000627, 10000629  10000600, 10000628            0.10

In [5]:
# mapping rules to relevant information.... 
print("mapping rules to relevant information....")
code_to_name_dict = pd.Series(df["ProductNameTH"].values, index=df["ProductCode"]).to_dict()
code_to_group_level1_dict = pd.Series(df["GroupNameLevel1"].values, index=df["ProductCode"]).to_dict()
code_to_group_level2_dict = pd.Series(df["GroupNameLevel2"].values, index=df["ProductCode"]).to_dict()
code_to_name_dict = {str(key): str(value) for key, value in code_to_name_dict.items()}
code_to_group_level1_dict = {str(key): str(value) for key, value in code_to_group_level1_dict.items()}
code_to_group_level2_dict = {str(key): str(value) for key, value in code_to_group_level2_dict.items()}
total_revenue_pdt_code_dict = df.groupby(['ProductCode'])['NetAmount'].sum().to_dict()
total_revenue_pdt_code_dict = {str(key): str(round(value, 2)) for key, value in total_revenue_pdt_code_dict.items()}


mapping rules to relevant information....


In [6]:

def updateRules(rules, dictUsed, colNameOut, colIndex, ant_con):
    for row in rules.itertuples():
        # basket = []
        print(str(row[i+1]).strip())
        if (ant_con == 1 and row[i+1] != None):
            # for pdt in list(row.antecedents.split(',')):
            pdt_name = dictUsed[str(row[i+1]).strip()]
            # basket.append(pdt_name)
            # rules.at[row.Index, colNameOut] = ', '.join(basket)
            rules.at[row.Index, colNameOut] = pdt_name
        elif (ant_con == 0 and row[i+1] != None):
            pdt_name = dictUsed[str(row[i+1]).strip()]
            print(pdt_name)
            rules.at[row.Index, colNameOut] = pdt_name

    return rules

# find and process antecedent columns
print("processing antecedent columns....")
print(rules.columns)
for i in range(len(rules.columns)-1):
    print(rules.columns[i])
    if "antecedent_" in rules.columns[i]:
        rules = updateRules(rules, code_to_name_dict, i, "ant_thai", 1)
        rules = updateRules(rules, code_to_group_level1_dict, "ant_grouplevel1", i, 1)
        rules = updateRules(rules, code_to_group_level2_dict, i, "ant_grouplevel2", 1)
        rules = updateRules(rules, total_revenue_pdt_code_dict, i, "ant_pdt_ttl_net_amt", 1)
    
    if "consequents_" in rules.columns[i]:
        rules = updateRules(rules, code_to_name_dict, i, "con_thai", 0)
        rules = updateRules(rules, code_to_group_level1_dict, i, "con_grouplevel1", 0)
        rules = updateRules(rules, code_to_group_level2_dict, i, "con_grouplevel2", 0)
        rules = updateRules(rules, total_revenue_pdt_code_dict, i, "con_pdt_ttl_net_amt", 0)


print(rules)

processing antecedent columns....
Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction'],
      dtype='object')
antecedents
consequents
antecedent support
consequent support
support
confidence
lift
leverage
                        antecedents           consequents  antecedent support  \
179            (10000602, 10000604)            (10000603)            0.098043   
178            (10000602, 10000603)            (10000604)            0.135761   
183                      (10000604)  (10000602, 10000603)            0.152707   
184            (10000627, 10000628)            (10000629)            0.142926   
189                      (10000629)  (10000627, 10000628)            0.152607   
180            (10000603, 10000604)            (10000602)            0.108376   
202            (10000600, 10000629)  (10000627, 10000628)            0.117806   
196  (10000600, 10000627, 10000628)           

In [7]:
# export rules to csv
print("exporting association rules to CSV....")
rules.to_csv("../ETL/CSV/association_rules.csv")

exporting association rules to CSV....
